In [2]:
!pip install azure-storage-blob --upgrade
!pip install azure-core


from azure.storage.blob import BlobServiceClient
import pandas as pd
import io

account_name = 'namehere'   
account_key = 'key here'
container_name = 'raw'
blob_name = 'dynamic_supply_chain_logistics_dataset.csv'


connection_string = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"

# Connect to Azure Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Download the blob
blob_client = container_client.get_blob_client(blob_name)
blob_data = blob_client.download_blob().readall()

# Load into pandas
df = pd.read_csv(io.BytesIO(blob_data))

# Check the data
print("✅ Data pulled successfully!")
print(df.head())


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
✅ Data pulled successfully!
             timestamp  vehicle_gps_latitude  vehicle_gps_longitude  \
0  2021-01-01 00:00:00             40.375568             -77.014318   
1  2021-01-01 01:00:00             33.507818            -117.036902   
2  2021-01-01 02:00:00             30.020640             -75.269224   
3  2021-01-01 03:00:00             36.649223             -70.190529   
4  2021-01-01 04:00:00             30.001279             -70.012195   

   fuel_consumption_rate  eta_variation_hours  traffic_congestion_level  \
0               5.136512             4.998009                  5.927586   
1               5.101512             0.984929                  1.591992   
2               5.090803             4.972665                  8.787765   
3               8.219558             3.095064                  0.045257   
4             

In [3]:


df_clean = df.dropna()


df_clean['loading_unloading_time'] = pd.to_numeric(df_clean['loading_unloading_time'], errors='coerce')
average_loading_time = df_clean['loading_unloading_time'].mean()
print(f"✅ Average Loading/Unloading Time: {average_loading_time:.2f} minutes")


def categorize_congestion(level):
    if level <= 2:
        return 'Low'
    elif level <= 5:
        return 'Moderate'
    else:
        return 'High'

df_clean['traffic_congestion_category'] = df_clean['traffic_congestion_level'].apply(categorize_congestion)

def stock_health(level):
    if level >= 70:
        return 'Healthy'
    elif level >= 40:
        return 'Moderate'
    else:
        return 'Critical'

df_clean['warehouse_stock_status'] = df_clean['warehouse_inventory_level'].apply(stock_health)


from datetime import datetime
df_clean['processing_timestamp'] = datetime.now()


local_transformed_file = "logistics_orders_transformed.csv"
df_clean.to_csv(local_transformed_file, index=False)

print("✅ ETL Transformations done and saved locally!")


✅ Average Loading/Unloading Time: 2.29 minutes
✅ ETL Transformations done and saved locally!
